In [1]:
def facility_path(instance):
    return 'documents/{}'.format(instance)

In [2]:
facility_path(12)

'documents/12'

In [ ]:
import docx
import pandas as pd
from os import path, listdir

In [ ]:
articles = pd.read_csv(r'jurnal_papers.csv')

In [ ]:
def read_docx(folder_path, folder_name):
    f_Text = []
    fText = ''
    try:
        for name in listdir(folder_path):
            if ((folder_name)== name[:-5]):
                filePath = path.join(folder_path, name)
                print('filePath is:{}'.format(filePath))
                doc = docx.Document(filePath)
                for line in doc.paragraphs:
                    f_Text.append(line.text)
            else:
                continue
    except:
        print("reading {} file failed, no such directory".format(folder_path))

    fText = '\n'.join(f_Text)
    return fText


In [ ]:
for i, a_id in enumerate(articles['folder']):
    link = r'D:\MWT\master_thesis\GCSAR\accepted\{}'.format(a_id)
    filetext = read_docx(link,a_id)
    articles.artc_txt[i] = filetext

In [ ]:
# articles.describe()
articles.head()

In [ ]:
import re
import string
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.utils.dediac import dediac_ar
from nltk.corpus import stopwords
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from farasa.stemmer import FarasaStemmer

In [ ]:
def remove_numbers_punctuations (text):
    clean_text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', ' ', str(text))
    clean_text = re.sub(r'\w*\d\w*', ' ', str(clean_text))
    clean_text = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', clean_text)
    clean_text = re.sub(r'،|؛', ' ', str(clean_text))
    return clean_text

In [ ]:
def tokenize(text):
    tokens = simple_word_tokenize(text)
    return tokens

In [ ]:
def remove_stop_words(tokens):
    filtered_text = [t for t in tokens if not t in stopwords.words("arabic")]
#     filtered_text = [t for t in filtered_text if not t in stopwords.words("english")]
    return filtered_text

In [ ]:
def remove_stop_words2(tokens):
    filtered_text = [t for t in tokens if not t in stopwords.words("english")]
    return filtered_text

In [ ]:
def ar_stemmer(text):
    stemmer = FarasaStemmer()
    stemmed_text = stemmer.stem(text)
    return stemmed_text


In [ ]:
def vectorizer(filtered_text):
    cv = CountVectorizer()
    transformed_text = cv.fit_transform(filtered_text)
    vector_table = pd.DataFrame(transformed_text.toarray(), columns=cv.get_feature_names())
    return vector_table

In [ ]:
def word_counter(word_list):
    counts = Counter(word_list)
    # Reverse the key/values in the dictionary for sorting
    word_counts = list(zip(counts.values(), counts.keys()))

    # Sort the list by count
    word_counts = sorted(word_counts, reverse=True)
    return word_counts

In [ ]:
def remove_single_double_char(text):
    no_singl_char =re.sub(r'[×©±≤≥]', ' ', str(text))
    no_singl_char =re.sub(r'\b\w{1,2}\b', ' ', str(no_singl_char))
#     simpols = '× , © , o , ≤ , ≥'
    return no_singl_char

In [ ]:
def only_ar_text(text):
    ar_text = re.sub(r"[A-Z]['\w]*|[a-z]['\w]*", ' ', str(text))
    return ar_text

In [ ]:
articles['clean_txt'] = articles.artc_txt.map(remove_numbers_punctuations)
# .map(tokenize).map(remove_stop_words)

In [ ]:
articles['clean_txt'] = articles['clean_txt'].map(tokenize)

In [ ]:
articles['clean_txt'] = articles['clean_txt'].map(remove_stop_words)

In [ ]:
articles.clean_txt[0:100]

In [ ]:
for i,txt in enumerate(articles.clean2_txt):
    articles.clean2_txt[i] = remove_stop_words2(articles.clean2_txt[i])
    print(i,'done')


In [ ]:
articles['clean_txt'] = articles['clean_txt'].map(remove_stop_words2)

In [ ]:
articles.tail()

In [ ]:
articles.to_csv(r'C:\Users\User\Dropbox\PC\Documents\python\14.csv' ,sep="#")

In [ ]:
joiner = lambda x : " ".join(x)

In [ ]:
articles['clean3_txt'] = articles['keywords']

In [ ]:
articles['clean3_txt'] = articles['clean2_txt'].map(joiner)

In [ ]:
articles['clean3_txt'] = articles['clean3_txt'].map(dediac_ar)

In [ ]:
articles['clean3_txt'] = articles['clean2_txt'].map(ar_stemmer)

In [ ]:
for i,txt in enumerate(articles.clean3_txt):
    articles.clean3_txt[i] = ar_stemmer(articles.clean3_txt[i])
    print(i,'done')


In [ ]:
articles['clean3_txt'] = articles['clean3_txt'].map(remove_single_double_char)

In [ ]:
articles['keywords'] = articles['clean3_txt'].map(tokenize).map(word_counter)

In [ ]:
articles['ar_keywords'] = articles['folder']

In [ ]:
for i,txt in enumerate(articles.ar_keywords):
    articles.ar_keywords[i] = only_ar_text(articles.keywords[i])
    print(i,'done')


In [ ]:
# saving models and dataframes
import joblib
# or from sklearn.externals import joblib

In [ ]:
joblib.dump(articles, "article_scope_model.pkl")

['article_scope_model.pkl']